In [55]:
%matplotlib inline
import torch
import numpy as np
import torchvision
from torch.utils import data
from torchvision import transforms


In [56]:
# 通过ToTensor实例将图像数据从PIL类型变换成32位浮点数格式，
# 并除以255使得所有像素的数值均在0到1之间
trans = transforms.ToTensor()
mnist_train = torchvision.datasets.FashionMNIST(
root="../data", train=True, transform=trans, download=True)
mnist_test = torchvision.datasets.FashionMNIST(
root="../data", train=False, transform=trans, download=False)

In [57]:
#数据读取
batch_size = 256
def get_dataloader_workers(): 
#"""使⽤4个进程来读取数据"""
    return 4
train_iter = data.DataLoader(mnist_train, batch_size, shuffle=True,
num_workers=get_dataloader_workers())
test_iter = data.DataLoader(mnist_test, batch_size, shuffle=True,
num_workers=get_dataloader_workers())

mnist_train[0][0].shape


torch.Size([1, 28, 28])

在这里，我们可以看到，每个数据都是 1 * 28 * 28 的形式，我们在这里，先将数据展平，每个数据有 28 * 28=784 个特征值。而输出共有10个特征。我们现在设计一层网络，即：输入-输出 网络，则权重为 784  * 10偏重为10 * 1，最终得到1 * 10的向量

In [58]:
def accuracy(y_hat, y): #@save
#"""计算预测正确的数量"""
    if len(y_hat.shape) > 1 and y_hat.shape[1] > 1:
        y_hat = y_hat.argmax(axis=1)
    cmp = y_hat.type(y.dtype) == y
    return float(cmp.type(y.dtype).sum())

In [63]:
from torch import nn
net=nn.Sequential(nn.Linear(784,10))
w=net[0].weight.data.normal_(0, 0.01)
b=net[0].bias.data.fill_(0)
loss=nn.CrossEntropyLoss()
sgd=torch.optim.SGD(net.parameters(), lr=0.03)


In [64]:
num_epochs=30
for epoch in range(num_epochs):
    a=[]
    for X,y in train_iter:
        y_hat=nn.functional.softmax(net(X.reshape((-1, w.shape[1]))))
        l=loss(y_hat,y)
        sgd.zero_grad()
        l.backward()
        sgd.step()
        ac=accuracy(y_hat,y)/len(y)
        a.append(ac)
    print(f'epoch {epoch + 1}, loss 从{a[0]}到{a[-1]}')

C:\Users\11215\AppData\Local\Temp/ipykernel_15456/3611611476.py:5: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  y_hat=nn.functional.softmax(net(X.reshape((-1, w.shape[1]))))


epoch 1, loss 从0.0625到0.625
epoch 2, loss 从0.625到0.6770833333333334
epoch 3, loss 从0.66796875到0.7604166666666666
epoch 4, loss 从0.703125到0.6770833333333334
epoch 5, loss 从0.67578125到0.65625
epoch 6, loss 从0.6875到0.6354166666666666
epoch 7, loss 从0.69921875到0.7291666666666666
epoch 8, loss 从0.7109375到0.71875
epoch 9, loss 从0.66796875到0.7395833333333334
epoch 10, loss 从0.71875到0.8020833333333334
epoch 11, loss 从0.7109375到0.7916666666666666
epoch 12, loss 从0.765625到0.75
epoch 13, loss 从0.75390625到0.7708333333333334
epoch 14, loss 从0.7734375到0.75
epoch 15, loss 从0.76171875到0.7916666666666666
epoch 16, loss 从0.76953125到0.7916666666666666
epoch 17, loss 从0.76953125到0.7708333333333334
epoch 18, loss 从0.7734375到0.78125
epoch 19, loss 从0.796875到0.6979166666666666
epoch 20, loss 从0.75390625到0.78125
epoch 21, loss 从0.77734375到0.7083333333333334
epoch 22, loss 从0.78125到0.84375
epoch 23, loss 从0.81640625到0.71875
epoch 24, loss 从0.75到0.78125
epoch 25, loss 从0.74609375到0.75
epoch 26, loss 从0.8125到0.7

In [65]:
class Accumulator: #@save
#"""在n个变量上累加"""
    def __init__(self, n):
        self.data = [0.0] * n
    def add(self, *args):
        self.data = [a + float(b) for a, b in zip(self.data, args)]
    def reset(self):
        self.data = [0.0] * len(self.data)
    def __getitem__(self, idx):
        return self.data[idx]

def evaluate_accuracy(net, data_iter): #@save
#"""计算在指定数据集上模型的精度"""
    if isinstance(net, torch.nn.Module):
        net.eval() # 将模型设置为评估模式
    metric = Accumulator(2) # 正确预测数、预测总数
    with torch.no_grad():
        for X, y in data_iter:
            metric.add(accuracy(net(X.reshape((-1, w.shape[1]))), y), y.numel())
    return metric[0] / metric[1]

print(evaluate_accuracy(net, test_iter))
print(evaluate_accuracy(net, train_iter))

0.775
0.7825833333333333
